In [ ]:
import pandas as pd
from datetime import datetime
import numpy as np
import torch
import torch.nn as nn

# Load the test data
data = pd.read_csv("Test.csv", index_col=0, header=[0,1], parse_dates=True)
da_test = torch.tensor(data["da"].values).float()
rt_test = torch.tensor(data["rt"].values).float()
X_test = torch.tensor(data["X"].values).float()

# Load your trained model (replace 'YourModel.pth' with the path to your saved model)
model = TradingModel(input_size, hidden_size, output_size)  # Make sure to use the same model architecture
model.load_state_dict(torch.load('YourModel.pth'))
model.eval()

# Test the model on the test data
with torch.no_grad():
    output = model(X_test)
    vl, vp, vs, sp = torch.split(output, 7, dim=1)
    profit = torch.sum((vl * torch.clamp_min(sp - da_test, min=0) + vs * torch.clamp_min(da_test - sp, min=0)) * (rt_test - da_test), dim=1)
    worst_loss, _ = torch.min(profit, dim=0)

print(f"Test profit: {profit.mean().item()}")
print(f"Worst loss: {-worst_loss.item()}")